In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head(2)

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(['EIN', 'NAME'], axis= 1, inplace = True)

#Drop noisy variables
application_df.drop(['SPECIAL_CONSIDERATIONS'], axis= 1, inplace = True)

In [3]:
app_counts = application_df["APPLICATION_TYPE"].value_counts()
replace_application = list(app_counts[app_counts < 100].index)
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: APPLICATION_TYPE, dtype: int64

In [4]:
class_counts = application_df["CLASSIFICATION"].value_counts()
replace_class = list(class_counts[class_counts < 1000].index)
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [5]:
application_cats = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
application_cats

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT']

In [6]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cats]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cats)

In [7]:
application_df = application_df.merge(encode_df,left_index=True, right_index=True)
application_df = application_df.drop(application_cats,1)
application_df.head()

C:\Users\menon\anaconda3\envs\PythonData\envs\mlenv\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [8]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

C:\Users\menon\anaconda3\envs\PythonData\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [29]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 8
hidden_nodes_layer2 = 8

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_14 (Dense)            (None, 8)                 344       
                                                                 
 dense_15 (Dense)            (None, 8)                 72        
                                                                 
 dense_16 (Dense)            (None, 1)                 9         
                                                                 
Total params: 425
Trainable params: 425
Non-trainable params: 0
_________________________________________________________________


In [30]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [31]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 1ms/step - loss: 14225.2363 - accuracy: 0.5097
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 10558.8242 - accuracy: 0.4942
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 9649.5957 - accuracy: 0.5052
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 9394.2090 - accuracy: 0.5188
Epoch 5/100
804/804 [==============================] - 1s 2ms/step - loss: 9093.8750 - accuracy: 0.5107
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 11639.2637 - accuracy: 0.4989
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 1082.6185 - accuracy: 0.5615
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 12133.4014 - accuracy: 0.5357
Epoch 9/100
804/804 [==============================] - 1s 1ms/step - loss: 7598.7397 - accuracy: 0.5175
Epoch 10/100
804/804 [==============================] - 1s 1

Epoch 81/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6909 - accuracy: 0.5335
Epoch 82/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6909 - accuracy: 0.5335
Epoch 83/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6909 - accuracy: 0.5335
Epoch 84/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6909 - accuracy: 0.5335
Epoch 85/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6910 - accuracy: 0.5335
Epoch 86/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6909 - accuracy: 0.5335
Epoch 87/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6909 - accuracy: 0.5335
Epoch 88/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6909 - accuracy: 0.5335
Epoch 89/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6910 - accuracy: 0.5335
Epoch 90/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6909

In [13]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [32]:
# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5)

In [33]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.8849 - accuracy: 0.6493 - 306ms/epoch - 1ms/step
Loss: 0.8849264979362488, Accuracy: 0.649329423904419


In [34]:
nn.save("AlphabetSoupCharityOptimization.h5")